# Linear Regression and the Least Squares Method

Linear regression is a fundamental technique in statistics and machine learning for modeling the relationship between a dependent variable $ y $ and one or more independent variables $ X $.

In this lab, we will:

1. Start with a simple linear regression example using real-world data.
2. Express the least squares method using matrix notation.
3. Perform multiple linear regression using built-in functions and matrix methods.
4. Explore what happens when the data matrix is not full rank and the least squares method becomes unstable.


## Simple Linear Regression with Real-World Data

We'll use the California Housing Prices dataset to predict house prices based on a single feature: the median income in the neighborhood.

The relationship between house price $( y )$ and median income $( X )$ is assumed to be linear:

$$
y = \beta_0 + \beta_1 X + \epsilon
$$

Our goal is to estimate the parameters $ \beta_0 $ (the intercept) and $ \beta_1 $ (the slope) using the least squares method. The symbol $\epsilon$ represents some random error.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.datasets import fetch_california_housing
import pandas as pd

# Load the dataset
housing = fetch_california_housing()
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['MedHouseVal'] = housing.target

# Filter out the observations where MedHouseVal == 5
df = df[df['MedHouseVal'] < 5]

# Select a random subset of the data (e.g., 1000 samples)
df_sample = df.sample(n=1000, random_state=42)

# Use only the median income as the predictor
X = df_sample[['MedInc']].values
y = df_sample['MedHouseVal'].values

# Plot the data
plt.scatter(X, y, alpha=0.5)
plt.title("House Prices vs Median Income (Filtered Subset)")
plt.xlabel("Median Income (in $10,000s)")
plt.ylabel("Median House Value (in $100,000s)")
plt.show()


## Least Squares Estimation

The least squares method aims to minimize the sum of the squared differences between the observed values $ y_i $ and the predicted values $ \hat{y_i}= \beta_0 + \beta_1 X_i$:

$$
\min_{\beta_0, \beta_1} \sum_{i=1}^{n} (y_i - (\beta_0 + \beta_1 X_i))^2
$$

Let's fit a simple linear regression model using the least squares method.


In [ ]:
# Fit the linear regression model
reg = LinearRegression()
reg.fit(X, y)

# Plot the regression line and the errors
plt.scatter(X, y, alpha=0.5)
plt.plot(X, reg.predict(X), color='red', label="Regression Line")
plt.title("Linear Regression Fit")
plt.xlabel("Median Income (in $10,000s)")
plt.ylabel("Median House Value (in $100,000s)")

# Plot the errors (vertical lines)
for i in range(len(X)):
    plt.plot([X[i], X[i]], [y[i], reg.predict(X)[i]], color='gray', alpha=0.5)

plt.legend()
plt.show()

# Display the coefficients
print(f"Intercept (β0): {reg.intercept_:.2f}")
print(f"Coefficient (β1): {reg.coef_[0]:.2f}")


## Matrix Formulation of Linear Regression

In matrix form, the linear regression model can be written as:

$$
y = X\beta + \epsilon
$$

where:
- $ y $ is the vector of observed values (in our case, the median house values).
- $ X $ is the design matrix, where each row corresponds to an observation, and each column corresponds to a predictor (including the intercept term).
- $ \beta $ is the vector of coefficients (including the intercept).
- $ \epsilon $ is the vector of errors.

### Understanding the Design Matrix $ X $

For a simple linear regression with one predictor (e.g., median income), the design matrix $ X $ consists of:
- A column of ones to account for the intercept term.
- A column representing the values of the predictor variable (in this case, median income).

Mathematically, for the first few observations, the design matrix $ X $ looks like this:

$$
X = \begin{pmatrix}
1 & X_1 \\
1 & X_2 \\
1 & X_3 \\
\vdots & \vdots \\
1 & X_n \\
\end{pmatrix}
= \begin{pmatrix}
1 & \text{MedInc}_1 \\
1 & \text{MedInc}_2 \\
1 & \text{MedInc}_3 \\
\vdots & \vdots \\
1 & \text{MedInc}_n \\
\end{pmatrix}
$$

where:
- The first column is all ones (corresponding to the intercept $ \beta_0 $).
- The second column consists of the values of the predictor variable (median income for each observation).


The least squares estimate of $ \beta $ is given by:

$$
\hat{\beta} = (X^TX)^{-1}X^Ty
$$

Let's compute this for our simple regression example.


In [ ]:
import numpy as np

# Add a column of ones to X for the intercept term
X_b = np.c_[np.ones((X.shape[0], 1)), X]

# Display the first few rows of the matrix X
print("First few rows of the design matrix X:")
print(X_b[:5])

# Compute the least squares solution using the formula
beta_hat = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print(f"Computed intercept (β0): {beta_hat[0]:.2f}")
print(f"Computed coefficient (β1): {beta_hat[1]:.2f}")


## Multiple Linear Regression

Now let's extend our example to multiple linear regression, where we have multiple predictor variables. The model can still be written as:

$$
y = X\beta + \epsilon
$$

We'll use additional features from the California Housing dataset (e.g., average rooms per dwelling, population, median income) to predict house prices.


In [ ]:
# Use multiple features to predict house prices
X_multi = df[['MedInc', 'AveRooms', 'AveOccup', 'HouseAge']].values
y_multi = df['MedHouseVal'].values

# Fit the linear regression model
reg_multi = LinearRegression()
reg_multi.fit(X_multi, y_multi)

# Display the coefficients
print(f"Intercept (β0): {reg_multi.intercept_:.2f}")
print(f"Coefficients (β1, β2, β3, β4): {reg_multi.coef_}")

# Matrix formulation
X_multi_b = np.c_[np.ones((X_multi.shape[0], 1)), X_multi]
beta_hat_multi = np.linalg.inv(X_multi_b.T.dot(X_multi_b)).dot(X_multi_b.T).dot(y_multi)

print(f"Computed intercept (β0): {beta_hat_multi[0]:.2f}")
print(f"Computed coefficients (β1, β2, β3, β4): {beta_hat_multi[1:]}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df_sample is the filtered and sampled dataframe from earlier

# List of predictors
predictors = ['MedInc', 'AveRooms', 'AveOccup', 'HouseAge']

# Create scatter plots
plt.figure(figsize=(15, 10))
for i, predictor in enumerate(predictors, 1):
    plt.subplot(2, 2, i)
    sns.scatterplot(x=df_sample[predictor], y=df_sample['MedHouseVal'], alpha=0.5)
    plt.title(f"House Prices vs {predictor}")
    plt.xlabel(predictor)
    plt.ylabel("Median House Value (in $100,000s)")

plt.tight_layout()
plt.show()
